In [1]:
# ricarica dei moduli src 
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

ROOT = Path.cwd().parent
SRC = ROOT / "src"

if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

from task1 import *
from task2 import *
from task3 import *

# Task 1

The dataset is structured across four merchandising levels, all organized under
a single umbrella hierarchy. The first level represents the macro-categories,
while the subsequent levels correspond to increasingly specific sub-categories.
The leaves of this hierarchy represent individual products. The goal of the first
task is to compute, for each merchandising level, the frequency of every
element. Then, for each level, you must create two bar plots: 1) one showing
the five most frequent elements, and 2)another showing the five least frequent
elements. Remember to exclude shoppers from the analysis. 

Carichiamo il dataset tramite la funzione *load_dataset()* e applichiamo la funzione *exclude_shoppers()* per rimuovere le righe associate agli articoli di tipo SHOPPERS

In [2]:
df = load_dataset()
df_clean = exclude_shoppers(df)

Definiamo un dizionario che mappa ciascun livello di merchandising alla relativa colonna descrittiva. Questo ci permette di iterare facilmente sui livelli della gerarchia dell'albero merceologico e generare in sequenza le successive analisi per ognuno di essi.

In [3]:
levels = {
    "liv1": "descr_liv1",
    "liv2": "descr_liv2",
    "liv3": "descr_liv3",
    "liv4": "descr_liv4"
}

Per ciascun livello di merchandising, richiamiamo la funzione *plot_frequency()* per generare i grafici delle frequenze per le categorie più e meno ricorrenti.

In [ ]:

# ---- Plotting ----
for level, desc_col in levels.items():
    plot_frequency(df_clean, desc_col, f"Level {level.upper()}")

# Task 2

The second task is similar to the first one, but requires stratifying the dataset.
The first stratification divides the dataset into three time periods based on
months:
◦ Range 1: January to Mid-May
◦ Range 2: Mid-May to September
◦ Range 3: October to December
For each of these ranges, you must create the same plots described in Task 1
for every merchandising level. The second stratification is based on time slots:
◦ Slot 1: 08:30–12:30
◦ Slot 2: 12:30–16:30
◦ Slot 3: 16:30–20:30
For each time slot and each merchandising level, you again need to generate
the same plots as in Task 1.


Definiamo una lista di intervalli temporali per poter suddividere il dataset.  
Ciascuno di essi è costituito da : 
- Nome dell'intervallo
- Data di inizio
- Data di fine

In [24]:
time_ranges = [
    ("Range 1: Gen - Metà Maggio", "2023-01-01", "2023-05-15"),
    ("Range 2: Metà Maggio - Settembre", "2023-05-16", "2023-09-30"),
    ("Range 3: Ott - Dic", "2023-10-01", "2023-12-31")
]

Per ciascun intervallo applichiamo un filtro sul dataframe con la funzione *filter_by_date_range(dataframe, start_date, end_date)* e poi esattamente come fatto in precedenza generiamo i grafici di frequenza per tutti i livelli di merchandising.

In [ ]:
for range_name, start_date, end_date in time_ranges:
    df_range = filter_by_date_range(df_clean, start_date, end_date) 
    for level, desc_col in levels.items():
        plot_frequency(df_range, desc_col, f"{range_name} - Level {level.upper()}")


Definiamo un'altra lista per poter suddividere il dataset in intervalli temporali orari.  
Ciascuno di essi è costituito da : 
- Nome dell'intervallo
- Ora di inizio
- Ora di fine

In [26]:
slot_ranges = [
    ("Range 1: 08:30–12:30", "08:30", "12:30"),
    ("Range 2: 12:30–16:30", "12:30", "16:30"),
    ("Range 3: 16:30–20:30", "16:30", "20:30")
]


Per ciascun intervallo applichiamo un filtro sul dataframe con la funzione *filter_by_hour_range(dataframe, start_hour, end_hour)* e poi esattamente come fatto in precedenza generiamo i grafici di frequenza per tutti i livelli di merchandising.

In [ ]:
for range_name, start_time, end_time in slot_ranges:
    df_slot = filter_by_hour_range(df_clean, start_time, end_time)
    for level, desc_col in levels.items():
        plot_frequency(df_slot, desc_col, f"{range_name} - {level.upper()}")


# Task 3

Prepariamo il formato richiesto dalla funzione **apriori()**  
Creiamo una matrice one-hot encoded con shape (n_transazioni, n_prodotti) dove ogni cella è 1 se quel prodotto è presente nella transazione, 0 altrimenti.

In [5]:
basket_sets = create_basket_sets(df_clean, "scontrino_id", "descr_liv4","r_qta_pezzi")


c:\Users\giova\Universita\data_mining\IDM_2025\first_classwork\src\task3.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_sets = basket.drop(transaction_col, axis=1).applymap(lambda x: 1 if x > 0 else 0)


Calcoliamo gli itemset frequenti con l'algoritmo Apriori tramite l'apposita funzione della libreria mlxtend.  
Per motivi relativi alla capacità della RAM configuriamo una soglia di supporto pari al 3% del numero di transazioni

In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)

c:\Users\giova\PythonEnvs\ml_env\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Calcoliamo le regole di associazione possibili e manteniamo solo quelle con una confidenza >= 0.3

In [7]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

### Analisi dei risultati ottenuti

Quanti itemset abbiamo ottenuto ?

In [ ]:
len(frequent_itemsets)

Visualizziamo i primi 10

In [ ]:
frequent_itemsets.sort_values('support', ascending=False).head(10)

Quante regole sono state estratte ?

In [18]:
len(rules)

55

Ordiniamo le regole per lift discendente e visualizziamo le 5 più interessanti

In [ ]:
rules_sorted = rules.sort_values('lift', ascending=False)
rules_sorted[['antecedents','consequents','support','confidence','lift']].head(5)

,antecedents,consequents,support,confidence,lift
30,(NORM.ASC.LUNGA),(NORM.ASC.CORTA),0.042447,0.552240,4.128536
31,(NORM.ASC.CORTA),(NORM.ASC.LUNGA),0.042447,0.317336,4.128536
15,(SUINO),(BOVINO),0.049003,0.496936,2.927894
49,(SALAME),(PASTE FILATE STAGIONATE),0.034670,0.352188,2.598991
43,(PREPARATI GASTRON.(PANETTERIA)),(PANETTERIA PANE),0.039576,0.672296,2.477000


# Task 4

Calcoliamo gli itemset frequenti con l'algoritmo FP Growth tramite l'apposita funzione della libreria mlxtend.  
Essendo un algoritmo più veloce ed efficiente di Apriori possiamo permetterci di abbassare la soglia di supporto rispetto a prima.

In [22]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(basket_sets, min_support=0.01, use_colnames=True)

c:\Users\giova\PythonEnvs\ml_env\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Calcoliamo le regole di associazione possibili e manteniamo solo quelle con una confidenza >= 0.3

In [23]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

### Analisi dei risultati ottenuti

Quanti itemset abbiamo ottenuto ?

In [24]:
len(frequent_itemsets)

1356

Visualizziamo i primi 10

In [25]:
frequent_itemsets.sort_values('support', ascending=False).head(10)

,support,itemsets
41,0.271415,(PANETTERIA PANE)
0,0.250793,(PROSCIUTTO COTTO)
46,0.249116,(NORMALE)
18,0.178327,(LATTE VACCINO)
1,0.173266,(NORMALI)
50,0.170446,(AVICUNICOLO)
89,0.169725,(BOVINO)
51,0.152936,(GRANA E SIMILI)
13,0.135958,(BANANE)
24,0.135510,(PASTE FILATE STAGIONATE)


Quante regole sono state estratte ?

In [26]:
len(rules)

855

Ordiniamo le regole per lift discendente e visualizziamo le 5 più interessanti

In [27]:
rules_sorted = rules.sort_values('lift', ascending=False)
rules_sorted[['antecedents','consequents','support','confidence','lift']].head(5)

,antecedents,consequents,support,confidence,lift
412,(PIATTI),(BICCHIERI),0.016237,0.408801,6.300594
371,"(NORMALI, NORM.ASC.CORTA)",(NORM.ASC.LUNGA),0.013632,0.365824,4.759360
388,"(NORM.ASC.LUNGA, BOTTIGLIA/VASO)",(NORM.ASC.CORTA),0.011546,0.633964,4.739499
392,"(NORMALE, NORM.ASC.CORTA)",(NORM.ASC.LUNGA),0.018459,0.360808,4.694099
375,"(LATTE VACCINO, NORM.ASC.LUNGA)",(NORM.ASC.CORTA),0.013658,0.626714,4.685304
